In [2]:
# Imports for this project
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns
from torch.utils import data
from tqdm import tqdm

In [3]:
# Grab the MNIST dataset
training_set = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
testing_set = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())

In [4]:
# Begin data preprocessing

def data_prepocessor(dataset):
    """
    Used to prepare the data for processing by normalizing the data and flattening the images.

    Args:
        dataset (torch.utils.data.Dataset): The dataset to preprocess.

    Returns:
        return_tensors Tuple[torch.Tensor, torch.Tensor]: A tuple containing the preprocessed data and the corresponding
    """

    new_data = (dataset.data / 255.0) - 0.5
    flattened_img_data = new_data.view(new_data.shape[0], -1)
    targets = dataset.targets

    return flattened_img_data, targets

x_train, y_train = data_prepocessor(training_set)
x_test, y_test = data_prepocessor(testing_set)

In [5]:
# Verify that GPU is connected and available

print(torch.__version__)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(f'The system GPU | {torch.cuda.get_device_name(0) if not None else 'Error'} | is {'AVAILABLE' if torch.cuda.is_available() else 'NOT AVAILABLE' }')

2.6.0+cu124
The system GPU | NVIDIA GeForce RTX 4060 Laptop GPU | is AVAILABLE


In [6]:
class MNIST_Auto_Encoder(nn.Module):
    def __init__(self, bottleneck, input_size=784, hidden_layer_size=784):
        super().__init__()

        layers = []

        layers.append(
            nn.Linear(in_features=input_size, out_features=hidden_layer_size)
        )

        layers.append(
            nn.ReLU()
        )

        layers.append(
            nn.Linear(in_features=hidden_layer_size, out_features=bottleneck)
        )

        layers.append(
            nn.ReLU()
        )

        layers.append(
            nn.Linear(in_features=bottleneck, out_features=hidden_layer_size)
        )

        self.hidden_layers = nn.Sequential(*layers)

        self.ouput_layer = nn.Linear(in_features=hidden_layer_size, out_features=input_size)

    def forward(self, x) -> torch.Tensor:
        x = self.hidden_layers(x)
        logits = self.ouput_layer(x)

        return logits
        

In [17]:
epochs = 50
batch_size = 64
learning_rate = 0.1

model = MNIST_Auto_Encoder(700, hidden_layer_size=1000)
model.to(device)
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

training_dataset = data.TensorDataset(x_train, y_train)

data_loader = data.DataLoader(
    training_dataset, 
    batch_size=batch_size, 
    shuffle=True,
    # num_workers=16,
    pin_memory=True
)

for epoch in range(epochs):
    print(f'----- Epoch: {epoch + 1}/{epochs} -----')

    avg_loss = 0

    for inputs, labels in tqdm(data_loader, desc='Training', unit='batch'):
        # Move data into the GPU
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Zero out gradients
        optimizer.zero_grad()

        # Calc loss
        logits = model(inputs)
        loss = loss_function(logits, labels)

        # Calc gradient and step
        loss.backward()
        optimizer.step()

        with torch.no_grad():
            avg_loss += loss.item()

    avg_loss = avg_loss / len(data_loader)
    print(f'   -> Loss: {avg_loss: .4f}\n')

----- Epoch: 1/50 -----


Training: 100%|██████████| 938/938 [00:03<00:00, 263.37batch/s]


   -> Loss:  0.5954

----- Epoch: 2/50 -----


Training: 100%|██████████| 938/938 [00:03<00:00, 286.18batch/s]


   -> Loss:  0.1777

----- Epoch: 3/50 -----


...

   -> Loss:  0.0719

----- Epoch: 6/50 -----


Training: 100%|██████████| 938/938 [00:03<00:00, 296.08batch/s]


   -> Loss:  0.0605

----- Epoch: 7/50 -----


Training:  61%|██████    | 572/938 [00:02<00:01, 273.12batch/s]


KeyboardInterrupt: 